In [3]:
from docx import Document
import re

def diagnose_placeholders(doc_path, replacements):
    doc = Document(doc_path)
    found_placeholders = set()
    missing_placeholders = set()
    mismatched_placeholders = {}

    # 正則表達式來匹配 {{xxx}} 格式的佔位符
    placeholder_pattern = r'\{\{(.+?)\}\}'

    for paragraph in doc.paragraphs:
        for match in re.finditer(placeholder_pattern, paragraph.text):
            placeholder = match.group(0)
            if placeholder in replacements:
                found_placeholders.add(placeholder)
            else:
                missing_placeholders.add(placeholder)

    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    for match in re.finditer(placeholder_pattern, paragraph.text):
                        placeholder = match.group(0)
                        if placeholder in replacements:
                            found_placeholders.add(placeholder)
                        else:
                            missing_placeholders.add(placeholder)

    # 檢查格式不匹配的佔位符
    for placeholder in replacements.keys():
        if placeholder not in found_placeholders and placeholder not in missing_placeholders:
            similar_placeholders = [p for p in found_placeholders.union(missing_placeholders) if p.lower() == placeholder.lower()]
            if similar_placeholders:
                mismatched_placeholders[placeholder] = similar_placeholders

    # 輸出診斷結果
    print(f"診斷結果 for {doc_path}:")
    print("成功匹配的佔位符:", found_placeholders)
    print("文檔中存在但未在替換字典中定義的佔位符:", missing_placeholders)
    print("替換字典中定義但在文檔中找不到的佔位符:", set(replacements.keys()) - found_placeholders)
    if mismatched_placeholders:
        print("可能存在大小寫或格式問題的佔位符:")
        for k, v in mismatched_placeholders.items():
            print(f"  預期: {k}, 文檔中發現: {v}")

def main():
    replacements = {
        '{{申請單編號}}': 'APR-20240628-01',
        '{{申請單位}}': '信用卡行企',
        '{{申請日期}}': '2024-06-29',
        '{{申請單位填表人}}': '蘇若甯',
        '{{變更類型}}': 'APR',
        '{{程式負責人}}': '優利/Daniel Chou',
        '{{預計完成日}}': '2024/06/27',
        '{{測試日期}}': '2024/06/27',
        '{{上線日}}': '2024/06/27',
        '{{進館通知函日期}}': '2024年06月27日',
        '{{內容說明}}': """
修改整體CTI資料庫查詢的效能優化後的問題：
1.	無法查詢NEW客戶資料。

""",
        '{{上線清單}}': """
[SQL]
•	[聯邦行企]240627-行企查不到NEW客戶-NEW部分欄位不能為null.sql

""",
        '{{過版流程}}': """
SQL執行：
使用SSMS連線至172.30.101.62 > DB[ ucti ]，開啟檔案全選執行=>
•	[聯邦行企]240627-行企查不到NEW客戶-NEW部分欄位不能為null.sql

""",
        
        '{{測試項目}}': """
修改整體CTI資料庫查詢的效能優化後的問題：
1.	無法查詢NEW客戶資料。
2.  其他。
""",
        '{{處理方法說明}}': """
問題修正：
1.	將該筆資料部分欄位不能為null，改為空字串。

""",
    }

    templates = [
        'template/設計變更申請單.docx',
        'template/系統過版申請單.docx',
        'template/系統測試表.docx',
        'template/聯邦銀行廠商進館上線函.docx'
    ]

    for template in templates:
        diagnose_placeholders(template, replacements)
        print("\n")

if __name__ == '__main__':
    main()

診斷結果 for template/設計變更申請單.docx:
成功匹配的佔位符: {'{{申請單編號}}', '{{申請單位}}', '{{申請單位填表人}}', '{{程式負責人}}', '{{預計完成日}}', '{{處理方法說明}}', '{{內容說明}}'}
文檔中存在但未在替換字典中定義的佔位符: set()
替換字典中定義但在文檔中找不到的佔位符: {'{{過版流程}}', '{{測試項目}}', '{{進館通知函日期}}', '{{上線清單}}', '{{變更類型}}', '{{測試日期}}', '{{上線日}}', '{{申請日期}}'}


診斷結果 for template/系統過版申請單.docx:
成功匹配的佔位符: {'{{過版流程}}', '{{申請單編號}}', '{{上線清單}}', '{{變更類型}}', '{{申請單位}}', '{{程式負責人}}', '{{上線日}}', '{{申請日期}}', '{{內容說明}}'}
文檔中存在但未在替換字典中定義的佔位符: set()
替換字典中定義但在文檔中找不到的佔位符: {'{{測試項目}}', '{{進館通知函日期}}', '{{申請單位填表人}}', '{{預計完成日}}', '{{測試日期}}', '{{處理方法說明}}'}


診斷結果 for template/系統測試表.docx:
成功匹配的佔位符: {'{{測試日期}}', '{{申請單編號}}', '{{測試項目}}', '{{申請單位}}'}
文檔中存在但未在替換字典中定義的佔位符: set()
替換字典中定義但在文檔中找不到的佔位符: {'{{過版流程}}', '{{進館通知函日期}}', '{{上線清單}}', '{{變更類型}}', '{{申請單位填表人}}', '{{程式負責人}}', '{{預計完成日}}', '{{上線日}}', '{{申請日期}}', '{{處理方法說明}}', '{{內容說明}}'}


診斷結果 for template/聯邦銀行廠商進館上線函.docx:
成功匹配的佔位符: {'{{進館通知函日期}}', '{{申請單編號}}', '{{申請單位}}'}
文檔中存在但未在替換字典中定義的佔位符: set()
替換字典中定義但在文檔中找不到的佔位符: {'{{過版流程}}', '{